<a href="https://colab.research.google.com/github/KevivJaknap/Story/blob/master/Copy_of_5_custom_gym_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Baselines Tutorial - Creating a custom Gym environment

Github repo: https://github.com/araffin/rl-tutorial-jnrr19

Stable-Baselines: https://github.com/hill-a/stable-baselines

Documentation: https://stable-baselines.readthedocs.io/en/master/

RL Baselines zoo: https://github.com/araffin/rl-baselines-zoo


## Introduction

In this notebook, you will learn how to use your own environment following the OpenAI Gym interface.
Once it is done, you can easily use any compatible (depending on the action space) RL algorithm from Stable Baselines on that environment.

## Install Dependencies and Stable Baselines Using Pip



In [2]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.0

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 248 kB 5.1 MB/s 
  Attempting uninstall: stable-baselines
    Found existing installation: stable-baselines 2.2.1
    Uninstalling stable-baselines-2.2.1:
      Successfully uninstalled stable-baselines-2.2.1


## First steps with the gym interface

As you have noticed in the previous notebooks, an environment that follows the gym interface is quite simple to use.
It provides to this user mainly three methods:
- `reset()` called at the beginning of an episode, it returns an observation
- `step(action)` called to take an action with the environment, it returns the next observation, the immediate reward, whether the episode is over and additional information
- (Optional) `render(method='human')` which allow to visualize the agent in action. Note that graphical interface does not work on google colab, so we cannot use it directly (we have to rely on `method='rbg_array'` to retrieve an image of the scene

Under the hood, it also contains two useful properties:
- `observation_space` which one of the gym spaces (`Discrete`, `Box`, ...) and describe the type and shape of the observation
- `action_space` which is also a gym space object that describes the action space, so the type of action that can be taken

The best way to learn about gym spaces is to look at the [source code](https://github.com/openai/gym/tree/master/gym/spaces), but you need to know at least the main ones:
- `gym.spaces.Box`: A (possibly unbounded) box in $R^n$. Specifically, a Box represents the Cartesian product of n closed intervals. Each interval has the form of one of [a, b], (-oo, b], [a, oo), or (-oo, oo). Example: A 1D-Vector or an image observation can be described with the Box space.
```python
# Example for using image as input:
observation_space = spaces.Box(low=0, high=255, shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)
```                                       

- `gym.spaces.Discrete`: A discrete space in $\{ 0, 1, \dots, n-1 \}$
  Example: if you have two actions ("left" and "right") you can represent your action space using `Discrete(2)`, the first action will be 0 and the second 1.



[Documentation on custom env](https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html)

In [ ]:
import gym

env = gym.make("CartPole-v1")

# Box(4,) means that it is a Vector with 4 components
print("Observation space:", env.observation_space)
print("Shape:", env.observation_space.shape)
# Discrete(2) means that there is two discrete actions
print("Action space:", env.action_space)

# The reset method is called at the beginning of an episode
obs = env.reset()
# Sample a random action
action = env.action_space.sample()
print("Sampled action:", action)
obs, reward, done, info = env.step(action)
# Note the obs is a numpy array
# info is an empty dict for now but can contain any debugging info
# reward is a scalar
print(obs.shape, reward, done, info)


Observation space: Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
Shape: (4,)
Action space: Discrete(2)
Sampled action: 1
(4,) 1.0 False {}


##  Gym env skeleton

In practice this is how a gym environment looks like.
Here, we have implemented a simple grid world were the agent must learn to go always left.

In [ ]:
import numpy as np
import gym
from gym import spaces


class GoLeftEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn to go always left. 
  """
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {'render.modes': ['console']}
  # Define constants for clearer code
  LEFT = 0
  RIGHT = 1

  def __init__(self, grid_size=10):
    super(GoLeftEnv, self).__init__()

    # Size of the 1D-grid
    self.grid_size = grid_size
    # Initialize the agent at the right of the grid
    self.agent_pos = grid_size - 1

    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    n_actions = 2
    self.action_space = spaces.Discrete(n_actions)
    # The observation will be the coordinate of the agent
    # this can be described both by Discrete and Box space
    self.observation_space = spaces.Box(low=0, high=self.grid_size,
                                        shape=(1,), dtype=np.float32)

  def reset(self):
    """
    Important: the observation must be a numpy array
    :return: (np.array) 
    """
    # Initialize the agent at the right of the grid
    self.agent_pos = self.grid_size - 1
    # here we convert to float32 to make it more general (in case we want to use continuous actions)
    return np.array([self.agent_pos]).astype(np.float32)

  def step(self, action):
    if action == self.LEFT:
      self.agent_pos -= 1
    elif action == self.RIGHT:
      self.agent_pos += 1
    else:
      raise ValueError("Received invalid action={} which is not part of the action space".format(action))

    # Account for the boundaries of the grid
    self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size)

    # Are we at the left of the grid?
    done = bool(self.agent_pos == 0)

    # Null reward everywhere except when reaching the goal (left of the grid)
    reward = 1 if self.agent_pos == 0 else 0

    # Optionally we can pass additional info, we are not using that for now
    info = {}

    return np.array([self.agent_pos]).astype(np.float32), reward, done, info

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # agent is represented as a cross, rest as a dot
    print("." * self.agent_pos, end="")
    print("x", end="")
    print("." * (self.grid_size - self.agent_pos))

  def close(self):
    pass
    

### Validate the environment

Stable Baselines provides a [helper](https://stable-baselines.readthedocs.io/en/master/common/env_checker.html) to check that your environment follows the Gym interface. It also optionally checks that the environment is compatible with Stable-Baselines (and emits warning if necessary).

In [ ]:
from stable_baselines.common.env_checker import check_env

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
env = GoLeftEnv()
# If the environment don't follow the interface, an error will be thrown
check_env(env, warn=True)

NameError: ignored

### Testing the environment

In [ ]:
env = GoLeftEnv(grid_size=10)

obs = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

GO_LEFT = 0
# Hardcoded best agent: always go left!
n_steps = 20
for step in range(n_steps):
  print("Step {}".format(step + 1))
  obs, reward, done, info = env.step(GO_LEFT)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render()
  if done:
    print("Goal reached!", "reward=", reward)
    break

.........x.
Box(0.0, 10.0, (1,), float32)
Discrete(2)
1
Step 1
obs= [8.] reward= 0 done= False
........x..
Step 2
obs= [7.] reward= 0 done= False
.......x...
Step 3
obs= [6.] reward= 0 done= False
......x....
Step 4
obs= [5.] reward= 0 done= False
.....x.....
Step 5
obs= [4.] reward= 0 done= False
....x......
Step 6
obs= [3.] reward= 0 done= False
...x.......
Step 7
obs= [2.] reward= 0 done= False
..x........
Step 8
obs= [1.] reward= 0 done= False
.x.........
Step 9
obs= [0.] reward= 1 done= True
x..........
Goal reached! reward= 1


### Try it with Stable-Baselines

Once your environment follow the gym interface, it is quite easy to plug in any algorithm from stable-baselines

In [ ]:
from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.common.cmd_util import make_vec_env

# Instantiate the env
env = GoLeftEnv(grid_size=10)
# wrap it
env = make_vec_env(lambda: env, n_envs=1)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
# Train the agent
model = ACKTR('MlpPolicy', env, verbose=1).learn(5000)

In [ ]:
# Test the trained agent
obs = env.reset()
n_steps = 20
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = env.step(action)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render(mode='console')
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward)
    break

## It is your turn now, be creative!

As an exercise, that's now your turn to build a custom gym environment.
There is no constrain about what to do, be creative! (but not too creative, there is not enough time for that)

If you don't have any idea, here is is a list of the environment you can implement:
- Transform the discrete grid world to a continuous one, you will need to change a bit the logic and the action space
- Create a 2D grid world and add walls
- Create a tic-tac-toe game


In [9]:
import numpy as np
import gym
from gym import spaces
import random

class TicTacToeEnv(gym.Env):

  def __init__(self):
    super(TicTacToeEnv, self).__init__()
    self.action_space = spaces.Discrete(9)
    self.observation_space = spaces.Box(low=-1, high=1,shape=(9,), dtype=np.int32)
    self.state = np.zeros([3,3], dtype=np.int32)

  def reset(self):
    self.state = np.zeros([3,3], dtype=np.int32)
    return np.zeros([3,3], dtype=np.int32).flatten()

  def step(self, action):

    get_action = lambda n: [n//3, n%3]
    # function to check win for both agents
    def check_win(state):
      # check horizontally
      for i in range(3):
        if state[i][0] == state[i][1] == state[i][2] != 0:
          return state[i][0]
      # check vertically
      for i in range(3):
        if state[0][i] == state[1][i] == state[2][i] != 0:
          return state[0][i]
      
      # check left-right diagonal
      if state[0][0] == state[1][1] == state[2][2] != 0:
        return state[0][0]
      
      #check right-left diagonal
      if state[2][0] == state[1][1] == state[0][2] != 0:
        return state[2][0]

      return 0

    reward = 0
    done = False

    # check if there are empty squares
    n_zeros = np.count_nonzero(self.state == 0)

    # if there are none then game is done
    if n_zeros == 0:
      done = True
    else:
      #play the move if possible else game done
      action_ = get_action(action)
      if self.state[int(action_[0])][int(action_[1])] == 0:
        self.state[int(action_[0])][int(action_[1])] = 1
      else:
        reward-=10
        done = True
    
    #check if agent has won
    win = check_win(self.state) == 1
    if win:
      reward = 10
      done = True 
    
    if not done:
      # get all indices of empty squares for random agent to play
      empty_squares = []
      for i in range(3):
        for j in range(3):
          if self.state[i][j] == 0:
            empty_squares.append([i,j])
      
      if len(empty_squares) == 0:
        # there are no empty squares
        done = True
      else:
        #play the move of random agent
        agent_move = random.choice(empty_squares)
        self.state[agent_move[0]][agent_move[1]] = -1

        #check if random agent has won
        win = check_win(self.state) == -1
        if win:
          reward = -5
          done = True 

    info = {}

    return self.state.flatten(), reward, done, info

  def render(self, mode='console'):
        if mode!= 'console':
          raise NotImplementedError()
        print(self.state)

  def close(self):
        pass

In [12]:
from stable_baselines.common.env_checker import check_env
env = TicTacToeEnv()
check_env(env, warn=True)

In [10]:
env = TicTacToeEnv()
for step in range(10):
  print("Step {}".format(step + 1))
  a = int(input())
  obs, reward, done, info = env.step(a)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render()
  if done:
    print("Goal reached!", "reward=", reward)
    break

Step 1
1
obs= [-1  1  0  0  0  0  0  0  0] reward= 0 done= False
[[-1  1  0]
 [ 0  0  0]
 [ 0  0  0]]
Step 2
4
obs= [-1  1  0  0  1  0  0  0 -1] reward= 0 done= False
[[-1  1  0]
 [ 0  1  0]
 [ 0  0 -1]]
Step 3
7
obs= [-1  1  0  0  1  0  0  1 -1] reward= 10 done= True
[[-1  1  0]
 [ 0  1  0]
 [ 0  1 -1]]
Goal reached! reward= 10


In [13]:
from stable_baselines import DQN, ACKTR
from stable_baselines.common.cmd_util import make_vec_env

env = TicTacToeEnv()
env = make_vec_env(lambda: env, n_envs=1)

In [14]:
model = ACKTR('MlpPolicy', env, verbose=1).learn(5000)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where














---------------------------------
| ep_len_mean        | 3.6      |
| ep_reward_mean     | -7       |
| explained_variance | 0.0364   |
| fps                | 12       |
| nupdates           | 1        |
| policy_entropy     | 2.2      |
| policy_loss        | -13.9    |
| total_timesteps    | 20       |
| value_loss         | 49.2     |
---------------------------------
---------------------------------
| ep_len_mean        | 3.07     |
| ep_reward_mean     | -6.05    |
| explained_variance | -0.27    |
| fps                | 373      |
| nupdates           | 100      |
| policy_entropy     | 2.11     |
| policy_loss        | -0.276   |
| total_timesteps    | 2000     |
| value_loss         | 4.07     |
---------------------------------
---------------------------------
| ep_len_mean        | 2.85     |
| ep_reward_mean     | 0.25     |
| explained_variance | 0.128    |
| fps                | 435      |
| nupdates           | 200      |
| policy_entropy     | 0.443    |
| policy_loss 

In [17]:
obs = env.reset()
n_steps = 20
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = env.step(action)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render(mode='console')
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward)
    break

Step 1
Action:  [4]
obs= [[ 0  0  0  0  1  0  0  0 -1]] reward= [0.] done= [False]
[[ 0  0  0]
 [ 0  1  0]
 [ 0  0 -1]]
Step 2
Action:  [3]
obs= [[ 0  0  0  1  1  0  0 -1 -1]] reward= [0.] done= [False]
[[ 0  0  0]
 [ 1  1  0]
 [ 0 -1 -1]]
Step 3
Action:  [5]
obs= [[0 0 0 0 0 0 0 0 0]] reward= [10.] done= [ True]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
Goal reached! reward= [10.]
